In [38]:
from ngsolve import *
from netgen.geom2d import SplineGeometry

from netgen.occ import *
import netgen.meshing as ngm
import numpy as np
from ngsolve.webgui import *
from netgen.geom2d import unit_square

def innerL2(gfu,mesh):
    return Integrate(InnerProduct(gfu,gfu),mesh)
def L2error(gfu,exact,mesh):
    return sqrt(innerL2(gfu-exact,mesh))

In [ ]:
A= -0.34012
a = 0.34898 
u_exact = IfPos(sqrt(x**2 + y**2)-a, A*log(sqrt(x**2+y**2)),sqrt(0.25-((x)**2+(y)**2)))

# generate a triangular mesh of mesh-size 0.2
geo = SplineGeometry()
geo.AddCircle(c=(0,0), r=2)
n = 4
mesh = Mesh(geo.GenerateMesh(maxh=0.5**n))
phi = IfPos(0.5-sqrt(x**2+y**2), sqrt(0.25-((x)**2+(y)**2)), 0)
max_iter = 13
#phi = CF(sqrt(0.25-(x-0.5)**2-(y-0.5)**2) if (x-0.5)**2+(y-0.5)**2 < 0.25 else 0)
alpha = Parameter(2) # step size 
# psi initial guess
psik = GridFunction
# proximal galerkin iterations
fesu = H1(mesh, order=12, dirichlet="bottom|right|top|left")
fesphi = L2(mesh, order=11) # L_inf 
fes = fesu*fesphi
 #bilinear form (not linear)
u, varphi = fes.TrialFunction()
v, psi = fes.TestFunction()


a = BilinearForm(fes)
a += alpha * grad(u) * grad(v)*dx + varphi*v*dx
a += u*psi*dx - exp(varphi)*psi*dx

psik = GridFunction(fesphi)
psik.Set(1)
eps = 1e-6
for k in range(1,max_iter):
    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)

    # linear form rhs
    l = LinearForm(fes)
    # f = 0
    l += (0+psik)*v*dx
    l += phi*psi*dx

    a.Assemble()
    l.Assemble()
    # No lifting given g = 0 on \partial\Omegagfu
    gfu = GridFunction(fes)
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    history = GridFunction(fes, multidim = 0)
    # here we may need to add another loop
    maxiter = 10
    print("Initializing Newton")
    for iter in range(maxiter):
        # Prints before the iteration: number of it, residual, energy
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        inv = a.mat.Inverse(freedofs = fes.FreeDofs())        
        w.data =  inv * res
        gfu.vec.data -= w
        history.AddMultiDimComponent(gfu.vec)
        if abs(InnerProduct(w,res)) < 1e-6:
            print("Converged")
            break
    
    print("Exiting newton with:", InnerProduct(w,res))
    # update psik
    psik.vec.data = gfu.components[1].vec
    print("Error L2 with exact solution at PG iteration:", k)
    print(L2error(exp(psik)+phi,u_exact,mesh))
    Draw(exp(psik)+ phi , mesh, "psi")
    Draw(u_exact, mesh, "u")


tilde_u = phi + exp(gfu.components[1])  # tilde_u > \phi



    



Iteration of proximal galerkin:  0
Initializing Newton


KeyboardInterrupt: 

In [ ]:
geo = SplineGeometry()
geo.AddCircle(c=(0,0), r=2)
n = 3
mesh = Mesh(geo.GenerateMesh(maxh=0.5**n))
phi = IfPos(0.5-sqrt(x**2+y**2), sqrt(0.25-((x)**2+(y)**2)), 0)
Draw(phi, mesh, "tilde_u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
# solve the simple laplacian example
fes = H1(mesh, order=2, dirichlet="bottom|right")
u, v = fes.TnT()
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
l = LinearForm(fes)
l += 0*v*dx
a.Assemble()
l.Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * l.vec
Draw(gfu, mesh, "u")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene